## Домашнее задание 3

Будем практиковаться на датасете недвижимости (sklearn.datasets.fetch_california_housing)

Ваша задача:

Создать Dataset для загрузки данных

Обернуть его в Dataloader

Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)

Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [135]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
from torch.nn.functional import normalize

Загрузим данные

In [136]:
df = fetch_california_housing()
type(df)

sklearn.utils.Bunch

In [137]:
import numpy as np
housing = np.array(df.data)

Признаки нужно нормировать. Наверняка делаю слишком сложно, но уж как умею

In [138]:
from sklearn import preprocessing
import pandas as pd

housing_df = pd.DataFrame(housing, columns=df.feature_names)
housing_df
scaler = preprocessing.MinMaxScaler()

names = housing_df.columns
d = scaler.fit_transform(housing)

scaled_df = pd.DataFrame(d, columns=names)
scaled_df.head()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,0.539668,0.784314,0.043512,0.020469,0.008941,0.001499,0.567481,0.211155
1,0.538027,0.392157,0.038224,0.018929,0.067210,0.001141,0.565356,0.212151
2,0.466028,1.000000,0.052756,0.021940,0.013818,0.001698,0.564293,0.210159
3,0.354699,1.000000,0.035241,0.021929,0.015555,0.001493,0.564293,0.209163
4,0.230776,1.000000,0.038534,0.022166,0.015752,0.001198,0.564293,0.209163


Обединяю нормализованные признаки с таргетом

In [139]:
concat_housing = pd.concat([scaled_df, pd.Series(df.target)], axis=1, ignore_index=True)

In [140]:
concat_housing.head(2)

,0,1,2,3,4,5,6,7,8
0,0.539668,0.784314,0.043512,0.020469,0.008941,0.001499,0.567481,0.211155,4.526
1,0.538027,0.392157,0.038224,0.018929,0.067210,0.001141,0.565356,0.212151,3.585


Бьем на обучающую и тестовую выборки

In [141]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_df, pd.Series(df.target), 
                                                    train_size=0.75, 
                                                    random_state=13)

In [142]:
torch_train = pd.concat([X_train, y_train], axis=1)
torch_train.head(2)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,0
5707,0.208101,0.686275,0.026242,0.022560,0.037949,0.001105,0.177471,0.609562,2.680
3754,0.170542,0.686275,0.024481,0.019684,0.034026,0.002061,0.174283,0.595618,1.902


In [143]:
torch_train.shape

(15480, 9)

Создаем класс МойДатасет

In [144]:
import torch

class MyDataset(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = torch.from_numpy(init_dataset)
        #self._base_dataset[:-1] = normalize(self._base_dataset[:-1], p=2.0, dim = 0)

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        data = self._base_dataset[idx][:-1]
        value = self._base_dataset[idx][-1]
        return data.float(), value.float()

Создаем MyDataset для обучающей выборки

In [145]:
torch_train = np.array(torch_train)
train_dataset = MyDataset(torch_train)

Пример 0-го элемента MyDataset

In [146]:
train_dataset[0]

(tensor([0.2081, 0.6863, 0.0262, 0.0226, 0.0379, 0.0011, 0.1775, 0.6096]),
 tensor(2.6800))

Проверяем как отработает len

In [147]:
len(train_dataset)

15480

Создаем loader для трейна

In [148]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)

Создадим тестовый МойДатасет и loader для него

In [149]:
torch_test = pd.concat([X_test, y_test], axis=1)
torch_test.head(2)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,0
2973,0.251921,0.274510,0.037906,0.019389,0.021441,0.002040,0.297556,0.524900,1.301
14432,0.187935,0.392157,0.026008,0.022022,0.028252,0.000955,0.027630,0.709163,2.294


In [151]:
torch_test = np.array(torch_test)
test_dataset = MyDataset(torch_test)

In [152]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8,\
                                          shuffle=False)

Сначала воспользуюсь сетью с прошлого домашнего задания (без BatchNorm и Dropout)

In [153]:
from torch import nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 4 * hidden_dim)
        self.fc2 = nn.Linear(4 * hidden_dim,  2 * hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        x = self.fc4(x)
        return x

net = Net(8, 100, 1)
net.train()

Net(
  (fc1): Linear(in_features=8, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=1, bias=True)
)

 Т.к. имеем дело с регрессией, в качестве функции ошибки используем MSE

 В качестве оптимизатора воспользуемся Adam

In [154]:
from torch import optim

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [155]:
num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]
        labels = labels.view(labels.shape[0], 1)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        #print(outputs)
        loss = criterion(outputs, labels).float()
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 300 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Train MSE: {loss.item():.4f}. ' \
                 )

            for i, test_data in enumerate(test_loader):
                test_inputs, test_labels = test_data[0], test_data[1]
                test_labels = test_labels.view(test_labels.shape[0], 1)
                test_outputs = net(test_data[0])
                #считаем и выводим тестовый лосс
                test_loss = criterion(test_outputs, test_labels).float()
            
            print(f'Test MSE: {test_loss.item():.3f}')
            net.train()

Epoch [1/10]. Train MSE: 6.2104. 
Test MSE: 1.800
Epoch [1/10]. Train MSE: 0.7148. 
Test MSE: 0.695
Epoch [2/10]. Train MSE: 0.4072. 
Test MSE: 0.252
Epoch [2/10]. Train MSE: 0.7834. 
Test MSE: 0.519
Epoch [3/10]. Train MSE: 0.3995. 
Test MSE: 0.305
Epoch [3/10]. Train MSE: 0.6140. 
Test MSE: 0.235
Epoch [4/10]. Train MSE: 0.5681. 
Test MSE: 0.379
Epoch [4/10]. Train MSE: 0.4321. 
Test MSE: 0.245
Epoch [5/10]. Train MSE: 0.4684. 
Test MSE: 0.224
Epoch [5/10]. Train MSE: 0.3959. 
Test MSE: 0.315
Epoch [6/10]. Train MSE: 0.4822. 
Test MSE: 0.333
Epoch [6/10]. Train MSE: 0.2932. 
Test MSE: 0.228
Epoch [7/10]. Train MSE: 0.4009. 
Test MSE: 0.212
Epoch [7/10]. Train MSE: 0.2744. 
Test MSE: 0.204
Epoch [8/10]. Train MSE: 0.4254. 
Test MSE: 0.237
Epoch [8/10]. Train MSE: 0.2744. 
Test MSE: 0.229
Epoch [9/10]. Train MSE: 0.5121. 
Test MSE: 0.202
Epoch [9/10]. Train MSE: 0.3046. 
Test MSE: 0.248
Epoch [10/10]. Train MSE: 0.2434. 
Test MSE: 0.191
Epoch [10/10]. Train MSE: 0.2470. 
Test MSE: 0.23

Оценим средний RMSE на всем тестовом датасете

In [156]:
net.eval()
cumul_loss = 0
batches_calc = 0
for i, test_data in enumerate(test_loader):
    test_inputs, test_labels = test_data[0], test_data[1]
    test_labels = test_labels.view(test_labels.shape[0], 1)
    test_outputs = net(test_data[0])
    test_loss = criterion(test_outputs, test_labels).float()
    test_loss = test_loss.detach().numpy()
    test_loss = np.sqrt(test_loss)
    cumul_loss += test_loss
    batches_calc +=1
  
print(f'AV TEST RMSE : {cumul_loss/batches_calc}')

AV TEST RMSE : 0.5898129984736442


Добавим в нейронную сеть слои BatchNorm и Dropout

In [157]:
from torch import nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 4 * hidden_dim)
        self.fc2 = nn.Linear(4 * hidden_dim,  2 * hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
        self.bn1 = nn.BatchNorm1d(4 * hidden_dim)
        self.bn2 = nn.BatchNorm1d(2 * hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)

    def forward(self, x):
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.dp(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.dp(x)
        x = self.bn2(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        x = self.dp(x)
        x = self.bn3(x)
        x = self.fc4(x)
        return x

net = Net(8, 100, 1)
net.train()

Net(
  (fc1): Linear(in_features=8, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=1, bias=True)
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp): Dropout(p=0.25, inplace=False)
)

Все остальные параметры оставим, как в сети выше

In [158]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [159]:
num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]
        labels = labels.view(labels.shape[0], 1)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        #print(outputs)
        loss = criterion(outputs, labels).float()
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 300 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Train MSE: {loss.item():.4f}. ' \
                 )

            for i, test_data in enumerate(test_loader):
                test_inputs, test_labels = test_data[0], test_data[1]
                test_labels = test_labels.view(test_labels.shape[0], 1)
                test_outputs = net(test_data[0])
                #считаем и выводим тестовый лосс
                test_loss = criterion(test_outputs, test_labels).float()
            
            print(f'Test MSE: {test_loss.item():.3f}')
            net.train()

Epoch [1/10]. Train MSE: 6.7932. 
Test MSE: 3.572
Epoch [1/10]. Train MSE: 0.8646. 
Test MSE: 0.402
Epoch [2/10]. Train MSE: 0.6021. 
Test MSE: 0.398
Epoch [2/10]. Train MSE: 0.9352. 
Test MSE: 0.261
Epoch [3/10]. Train MSE: 0.5630. 
Test MSE: 0.221
Epoch [3/10]. Train MSE: 0.3603. 
Test MSE: 0.322
Epoch [4/10]. Train MSE: 0.4887. 
Test MSE: 0.257
Epoch [4/10]. Train MSE: 0.8701. 
Test MSE: 0.263
Epoch [5/10]. Train MSE: 0.6056. 
Test MSE: 0.570
Epoch [5/10]. Train MSE: 0.9756. 
Test MSE: 0.250
Epoch [6/10]. Train MSE: 0.4365. 
Test MSE: 0.245
Epoch [6/10]. Train MSE: 0.5159. 
Test MSE: 0.277
Epoch [7/10]. Train MSE: 0.5191. 
Test MSE: 0.208
Epoch [7/10]. Train MSE: 0.5535. 
Test MSE: 0.233
Epoch [8/10]. Train MSE: 0.4249. 
Test MSE: 0.226
Epoch [8/10]. Train MSE: 0.4693. 
Test MSE: 0.376
Epoch [9/10]. Train MSE: 0.3519. 
Test MSE: 0.253
Epoch [9/10]. Train MSE: 0.3180. 
Test MSE: 0.205
Epoch [10/10]. Train MSE: 0.3074. 
Test MSE: 0.261
Epoch [10/10]. Train MSE: 0.4315. 
Test MSE: 0.22

Оценим средний RMSE на тестовом датасете

In [160]:
net.eval()
cumul_loss = 0
batches_calc = 0
for i, test_data in enumerate(test_loader):
    test_inputs, test_labels = test_data[0], test_data[1]
    test_labels = test_labels.view(test_labels.shape[0], 1)
    test_outputs = net(test_data[0])
    test_loss = criterion(test_outputs, test_labels).float()
    test_loss = test_loss.detach().numpy()
    test_loss = np.sqrt(test_loss)
    cumul_loss += test_loss
    batches_calc +=1
  
print(f'AV TEST RMSE : {cumul_loss/batches_calc}')

AV TEST RMSE : 0.6281284101480662


Судя по данному примеру для этой задаче добавление BatchNorm и Dropout ухудшило качество модели. По факту переобучения не было и без этих слоев, поэтому их применение в принципе было не нужно здесь.

Теперь попробуем менять оптимизаторы

In [161]:
from torch import nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 4 * hidden_dim)
        self.fc2 = nn.Linear(4 * hidden_dim,  2 * hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
        self.bn1 = nn.BatchNorm1d(4 * hidden_dim)
        self.bn2 = nn.BatchNorm1d(2 * hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)

    def forward(self, x):
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.dp(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.dp(x)
        x = self.bn2(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        x = self.dp(x)
        x = self.bn3(x)
        x = self.fc4(x)
        return x

net = Net(8, 100, 1)
net.train()

Net(
  (fc1): Linear(in_features=8, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=1, bias=True)
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp): Dropout(p=0.25, inplace=False)
)

В качестве оптимизатора воспользуемся SGD с momentum = 0.9

In [162]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [163]:
num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]
        labels = labels.view(labels.shape[0], 1)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        #print(outputs)
        loss = criterion(outputs, labels).float()
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 300 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Train MSE: {loss.item():.4f}. ' \
                 )

            for i, test_data in enumerate(test_loader):
                test_inputs, test_labels = test_data[0], test_data[1]
                test_labels = test_labels.view(test_labels.shape[0], 1)
                test_outputs = net(test_data[0])
                #считаем и выводим тестовый лосс
                test_loss = criterion(test_outputs, test_labels).float()
            
            print(f'Test MSE: {test_loss.item():.3f}')
            net.train()

Epoch [1/10]. Train MSE: 5.9006. 
Test MSE: 3.063
Epoch [1/10]. Train MSE: 0.3644. 
Test MSE: 0.355
Epoch [2/10]. Train MSE: 0.7125. 
Test MSE: 0.339
Epoch [2/10]. Train MSE: 0.6847. 
Test MSE: 0.480
Epoch [3/10]. Train MSE: 1.0898. 
Test MSE: 0.263
Epoch [3/10]. Train MSE: 0.7159. 
Test MSE: 0.282
Epoch [4/10]. Train MSE: 0.8529. 
Test MSE: 0.210
Epoch [4/10]. Train MSE: 0.3539. 
Test MSE: 0.256
Epoch [5/10]. Train MSE: 0.7043. 
Test MSE: 0.303
Epoch [5/10]. Train MSE: 0.5239. 
Test MSE: 0.559
Epoch [6/10]. Train MSE: 0.5431. 
Test MSE: 0.293
Epoch [6/10]. Train MSE: 0.7900. 
Test MSE: 0.253
Epoch [7/10]. Train MSE: 0.6798. 
Test MSE: 0.198
Epoch [7/10]. Train MSE: 0.4829. 
Test MSE: 0.407
Epoch [8/10]. Train MSE: 0.5798. 
Test MSE: 0.290
Epoch [8/10]. Train MSE: 0.5570. 
Test MSE: 0.401
Epoch [9/10]. Train MSE: 0.3970. 
Test MSE: 0.262
Epoch [9/10]. Train MSE: 0.6185. 
Test MSE: 0.390
Epoch [10/10]. Train MSE: 0.7627. 
Test MSE: 0.318
Epoch [10/10]. Train MSE: 0.5570. 
Test MSE: 0.36

Оценим средний RMSE на тестовом датасете

In [164]:
net.eval()
cumul_loss = 0
batches_calc = 0
for i, test_data in enumerate(test_loader):
    test_inputs, test_labels = test_data[0], test_data[1]
    test_labels = test_labels.view(test_labels.shape[0], 1)
    test_outputs = net(test_data[0])
    test_loss = criterion(test_outputs, test_labels).float()
    test_loss = test_loss.detach().numpy()
    test_loss = np.sqrt(test_loss)
    cumul_loss += test_loss
    batches_calc +=1
  
print(f'AV TEST RMSE : {cumul_loss/batches_calc}')

AV TEST RMSE : 0.6364794627871624


В данном случае SGD отработал чуть-чуть хуже Adam

Построим еще одну сеть с оптимизатором RMSProp

In [165]:
from torch import nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 4 * hidden_dim)
        self.fc2 = nn.Linear(4 * hidden_dim,  2 * hidden_dim)
        self.fc3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
        self.bn1 = nn.BatchNorm1d(4 * hidden_dim)
        self.bn2 = nn.BatchNorm1d(2 * hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)

    def forward(self, x):
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.dp(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.dp(x)
        x = self.bn2(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        x = self.dp(x)
        x = self.bn3(x)
        x = self.fc4(x)
        return x

net = Net(8, 100, 1)
net.train()

Net(
  (fc1): Linear(in_features=8, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=1, bias=True)
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp): Dropout(p=0.25, inplace=False)
)

In [166]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(net.parameters(), lr=0.01)

In [167]:
num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]
        labels = labels.view(labels.shape[0], 1)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        #print(outputs)
        loss = criterion(outputs, labels).float()
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 300 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Train MSE: {loss.item():.4f}. ' \
                 )

            for i, test_data in enumerate(test_loader):
                test_inputs, test_labels = test_data[0], test_data[1]
                test_labels = test_labels.view(test_labels.shape[0], 1)
                test_outputs = net(test_data[0])
                #считаем и выводим тестовый лосс
                test_loss = criterion(test_outputs, test_labels).float()
            
            print(f'Test MSE: {test_loss.item():.3f}')
            net.train()

Epoch [1/10]. Train MSE: 5.2337. 
Test MSE: 0.905
Epoch [1/10]. Train MSE: 0.9475. 
Test MSE: 0.248
Epoch [2/10]. Train MSE: 0.8529. 
Test MSE: 0.421
Epoch [2/10]. Train MSE: 0.5545. 
Test MSE: 0.405
Epoch [3/10]. Train MSE: 0.4534. 
Test MSE: 0.297
Epoch [3/10]. Train MSE: 0.6490. 
Test MSE: 0.243
Epoch [4/10]. Train MSE: 0.6857. 
Test MSE: 0.314
Epoch [4/10]. Train MSE: 0.2769. 
Test MSE: 0.259
Epoch [5/10]. Train MSE: 0.4475. 
Test MSE: 0.305
Epoch [5/10]. Train MSE: 0.4688. 
Test MSE: 0.276
Epoch [6/10]. Train MSE: 0.7133. 
Test MSE: 0.331
Epoch [6/10]. Train MSE: 0.3413. 
Test MSE: 0.247
Epoch [7/10]. Train MSE: 0.6592. 
Test MSE: 0.343
Epoch [7/10]. Train MSE: 0.5261. 
Test MSE: 0.313
Epoch [8/10]. Train MSE: 0.4471. 
Test MSE: 0.467
Epoch [8/10]. Train MSE: 0.4210. 
Test MSE: 0.267
Epoch [9/10]. Train MSE: 0.3966. 
Test MSE: 0.256
Epoch [9/10]. Train MSE: 0.5526. 
Test MSE: 0.300
Epoch [10/10]. Train MSE: 0.3301. 
Test MSE: 0.245
Epoch [10/10]. Train MSE: 0.9319. 
Test MSE: 0.29

Оценим средний RMSE на тестовом датасете

In [168]:
net.eval()
cumul_loss = 0
batches_calc = 0
for i, test_data in enumerate(test_loader):
    test_inputs, test_labels = test_data[0], test_data[1]
    test_labels = test_labels.view(test_labels.shape[0], 1)
    test_outputs = net(test_data[0])
    test_loss = criterion(test_outputs, test_labels).float()
    test_loss = test_loss.detach().numpy()
    test_loss = np.sqrt(test_loss)
    cumul_loss += test_loss
    batches_calc +=1
  
print(f'AV TEST RMSE : {cumul_loss/batches_calc}')

AV TEST RMSE : 0.7672419993914374


В данном примере RMSProp отработал хуже других